# Testing Environment

In [1]:
print("hello")

hello


In [2]:
import torch
print("pytorch")

x = torch.rand(5, 3)
print(x)

pytorch
tensor([[0.8646, 0.7191, 0.3828],
        [0.3790, 0.3049, 0.8329],
        [0.7769, 0.3626, 0.5144],
        [0.2469, 0.7711, 0.4298],
        [0.1972, 0.9742, 0.5497]])


In [3]:
torch.cuda.is_available()

True

# 0

In [5]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%
100.0%
100.0%
100.0%


In [7]:
batch_size = 64


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [9]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

model.train()

Epoch 1
-------------------------------
loss: 1.176637  [   64/60000]
loss: 1.166381  [ 6464/60000]
loss: 0.986274  [12864/60000]
loss: 1.114092  [19264/60000]
loss: 1.012665  [25664/60000]
loss: 1.027703  [32064/60000]
loss: 1.060163  [38464/60000]
loss: 1.004546  [44864/60000]
loss: 1.047902  [51264/60000]
loss: 0.969414  [57664/60000]
Test Error: 
 Accuracy: 65.5%, Avg loss: 0.990255 

Epoch 2
-------------------------------
loss: 1.051432  [   64/60000]
loss: 1.062118  [ 6464/60000]
loss: 0.865741  [12864/60000]
loss: 1.016534  [19264/60000]
loss: 0.920640  [25664/60000]
loss: 0.930006  [32064/60000]
loss: 0.978278  [38464/60000]
loss: 0.927226  [44864/60000]
loss: 0.966601  [51264/60000]
loss: 0.898757  [57664/60000]
Test Error: 
 Accuracy: 66.8%, Avg loss: 0.914499 

Epoch 3
-------------------------------
loss: 0.959666  [   64/60000]
loss: 0.990522  [ 6464/60000]
loss: 0.780647  [12864/60000]
loss: 0.948453  [19264/60000]
loss: 0.859986  [25664/60000]
loss: 0.859316  [32064/600

#### Saving and using models

In [15]:
torch.save(model.state_dict(), "models/modelZero.pth")
print("Saved PyTorch Model State to models/modelZero.pth")

Saved PyTorch Model State to models/modelZero.pth


In [19]:
modeltwo = NeuralNetwork().to(device)
modeltwo.load_state_dict(torch.load("models/modelZero.pth", weights_only=True))

<All keys matched successfully>

#### using a class to make predictions

In [20]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Tensors

In [5]:
import torch
import numpy as np

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data)
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])


In [7]:
x_ones = torch.ones_like(x_data)
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(x_ones)
print(x_rand)

tensor([[1, 1],
        [1, 1]])
tensor([[0.7254, 0.5776],
        [0.6088, 0.0166]])


In [8]:
shape = (2,3,)
zeros_tensor = torch.zeros(shape)
zeros_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [9]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [10]:
if torch.accelerator.is_available():
    tensor = tensor.to(torch.accelerator.current_accelerator())
    
    
tensor.device

device(type='cuda', index=0)

In [6]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
tensor[2,:] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [1., 0., 1., 1.]])


In [ ]:
t1 = torch.cat([tensor, tensor, tensor], dim=0)
t2 = torch.cat([t1, t1, t1], dim=1)
print(t2)

In [22]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
# ``tensor.T`` returns the transpose of a tensor
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)
print(y1)
print(z1)

tensor([[3., 3., 0., 3.],
        [3., 3., 0., 3.],
        [0., 0., 0., 0.],
        [3., 3., 0., 3.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [0., 0., 0., 0.],
        [1., 0., 1., 1.]])


In [7]:
agg = tensor.sum()
print(agg)
agg_item = agg.item()
print(agg_item, type(agg_item))

tensor[2,3].item()

tensor(9.)
9.0 <class 'float'>


0.0

In [12]:
print(tensor)
tensor.add_(5)
tensor

tensor([[21., 20., 21., 21.],
        [21., 20., 21., 21.],
        [20., 20., 20., 20.],
        [21., 20., 21., 21.]])


tensor([[26., 25., 26., 26.],
        [26., 25., 26., 26.],
        [25., 25., 25., 25.],
        [26., 25., 26., 26.]])

# Neural Networks

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")



Using cuda device


Defining neural network subclass.

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [3]:
model = NeuralNetwork().to(device)
print(model)
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Predicted class: tensor([6], device='cuda:0')


In [4]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


Flatten converts 28x28 pixels into 784 straight pixels

In [5]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### Linear transformation using stored weights

In [6]:
#                              784 
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### non-linear activations

In [7]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2863, -0.1835,  0.2807,  0.2506, -0.4539,  0.3146, -0.2696,  0.2791,
          0.7915,  0.1106, -0.0394,  0.1902, -0.3022, -0.3179,  0.4184, -0.2890,
          0.7596, -0.0509,  0.2559, -0.0418],
        [ 0.6237,  0.4098, -0.1824, -0.2338, -0.1675,  0.3175,  0.0223, -0.0710,
          0.3594,  0.3456,  0.0398,  0.5273, -0.4998, -0.2923,  0.4446, -0.2518,
          0.8185, -0.4734,  0.5890,  0.2801],
        [ 0.1291,  0.1488, -0.3430, -0.0763, -0.0506,  0.1337, -0.3828, -0.0299,
          0.4809,  0.0268, -0.1877,  0.4401, -0.4691, -0.1501,  0.3583, -0.1596,
          0.8956, -0.2663,  0.1668,  0.0937]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2863, 0.0000, 0.2807, 0.2506, 0.0000, 0.3146, 0.0000, 0.2791, 0.7915,
         0.1106, 0.0000, 0.1902, 0.0000, 0.0000, 0.4184, 0.0000, 0.7596, 0.0000,
         0.2559, 0.0000],
        [0.6237, 0.4098, 0.0000, 0.0000, 0.0000, 0.3175, 0.0223, 0.0000, 0.3594,
         0.3456, 0.0398, 0.5273, 0.0000, 0.0000, 0.44

#### stacking operations

In [8]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

The last linear layer of the neural network returns logits: raw values in [-infinity infinity]
We can find the probability of these logits with softmax

In [9]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(logits)
print(pred_probab)

tensor([[ 0.1024, -0.3568, -0.0402,  0.1342,  0.0029,  0.0941, -0.0387, -0.1096,
         -0.0488,  0.1580],
        [-0.1179, -0.2215, -0.3827,  0.0153,  0.1636,  0.2286, -0.0416, -0.2326,
         -0.1104,  0.1343],
        [ 0.0493, -0.2974, -0.1828,  0.2305, -0.0268,  0.1896, -0.2025, -0.1624,
         -0.2120,  0.1834]], grad_fn=<AddmmBackward0>)
tensor([[0.1108, 0.0700, 0.0961, 0.1144, 0.1003, 0.1099, 0.0962, 0.0897, 0.0953,
         0.1172],
        [0.0925, 0.0834, 0.0709, 0.1056, 0.1225, 0.1307, 0.0998, 0.0824, 0.0931,
         0.1190],
        [0.1078, 0.0762, 0.0855, 0.1292, 0.0999, 0.1240, 0.0838, 0.0872, 0.0830,
         0.1233]], grad_fn=<SoftmaxBackward0>)


### Parameters

In [10]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0088,  0.0257, -0.0034,  ..., -0.0149, -0.0054,  0.0101],
        [ 0.0011,  0.0342,  0.0223,  ..., -0.0243, -0.0327, -0.0089]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0280,  0.0020], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0121,  0.0324, -0.0320,  ...,  0.0076,  0.0126,  0.0408],
        [-0.0239,  0.0194, -0.0427,  ...,  0.0349, -0.0229,  0.0238]],
       device='cuda:0', grad_fn=<Sl